# Using the Video Intelligence API with Python

Google Cloud's Video Intelligence API is a cloud service that allows various analyses to be performed on videos. The Video Intelligence API automatically processes videos, extracting meaning from the content, providing labeling, recognizing content, conducting sentiment analysis, and transcribing video to text. In this article, we discuss the usage of the Video Intelligence API with Python, demonstrating the transcription of audio from a video and performing sentiment analysis using TextBlob



In [ ]:
#Install the videointelligenceapi library.
!pip install google-cloud-videointelligence

Google Cloud SDK'yı yükleyiniz. (https://cloud.google.com/sdk/docs/install).

In [ ]:
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS']='videoanalysis-413611-07e890a20612.json'

In [ ]:
from datetime import timedelta
from typing import Optional, Sequence, cast

from google.cloud import videointelligence_v1 as vi


def transcribe_speech(
    video_uri: str,
    language_code: str,
    segments: Optional[Sequence[vi.VideoSegment]] = None,
) -> vi.VideoAnnotationResults:
    video_client = vi.VideoIntelligenceServiceClient()
    features = [vi.Feature.SPEECH_TRANSCRIPTION]
    config = vi.SpeechTranscriptionConfig(
        language_code=language_code,
        enable_automatic_punctuation=True,
    )
    context = vi.VideoContext(
        segments=segments,
        speech_transcription_config=config,
    )
    request = vi.AnnotateVideoRequest(
        input_uri=video_uri,
        features=features,
        video_context=context,
    )

    print(f'Processing video "{video_uri}"...')
    operation = video_client.annotate_video(request)

    # Wait for operation to complete
    response = cast(vi.AnnotateVideoResponse, operation.result())
    # A single video is processed
    results = response.annotation_results[0]

    return results

In [ ]:
#Paste the link of the video you uploaded to the Cloud Storage section into the video_uri section.

video_uri = "gs://videobucketproject/4K Ostrich the Flightless Bird - African Wildlife Documentary Film with Narration.mp4"
language_code = "en-US"
segment = vi.VideoSegment(
    start_time_offset=timedelta(seconds=1), #Specify the start and end times in seconds.
    end_time_offset=timedelta(seconds=220),
)

results = transcribe_speech(video_uri, language_code, [segment])

In [ ]:
def print_video_speech(results: vi.VideoAnnotationResults, min_confidence: float = 0.8):
    def keep_transcription(transcription: vi.SpeechTranscription) -> bool:
        return min_confidence <= transcription.alternatives[0].confidence

    transcriptions = results.speech_transcriptions
    transcriptions = [t for t in transcriptions if keep_transcription(t)]

    print(f" Speech transcriptions: {len(transcriptions)} ".center(80, "-"))
    for transcription in transcriptions:
        first_alternative = transcription.alternatives[0]
        confidence = first_alternative.confidence
        transcript = first_alternative.transcript
        print(f" {confidence:4.0%} | {transcript.strip()}")


In [ ]:
print(print_video_speech(results))

In [ ]:
from textblob import TextBlob

def analyze_sentiment(transcript):
    blob = TextBlob(transcript)
    sentiment_score = blob.sentiment.polarity
    sentiment_label = "Positive" if sentiment_score > 0 else "Negative" if sentiment_score < 0 else "Neutral"

    return sentiment_label, sentiment_score


transcripts = [
    "On the Open Spaces of the Kalahari Desert it's hard to find a place for such a high concentration of animals can be seen water holes and pens which are crucial sources of drinking attract wild animals such as these amazing envelope..",
    "I'm on the small herd of envelope. You can see a huge long-legged bird with a matching long neck male African ostrich. It's peering into the distance and evaluating the situation standing more than a feet tall with a secure site the Ostrich can see a danger at the distance of many miles.",
    "It's family is waiting for a nearby is trying to drink water cautiously bending its head to the ground but constantly looking around its surroundings. Maybe it's all Predators finally the Ostrich took the first sip of water while raising the head to look around one more time and let's water get to it's too just attract through the long oesophagus.",
    "After making sure that the place is safe in water is tasty the meal returns to its offspring two females are waiting patiently looking after their chicks after reporting the situation by the water the head of the family invites others to follow it.",
    "Chicks born different females are members of the scene family..."
]

for i, transcript in enumerate(transcripts, 1):
    sentiment_label, sentiment_score = analyze_sentiment(transcript)
    print(f"Transcript {i}: {transcript}")
    print("Sentiment Label:", sentiment_label)
    print("Sentiment Score:", sentiment_score)
    print("\n")
